In [1]:
#Part B - Alternate - Structured Streaming 
#Come up with a set of at least 10 SQL questions that involve joins, order by, group by and aggregate statements and implement them on your data using Spark SQL.
em= spark.read.csv('/FileStore/tables/data.csv', header = True, inferSchema = True)
features=spark.read.csv('/FileStore/tables/Features.csv', header = True, inferSchema = True)
features = features.selectExpr("Fuel_Price as price","Store as Store","CPI as CPI","Unemployment as Unemployment","IsHoliday as IsHoliday","Temperature as Temperature")
sales=spark.read.csv('/FileStore/tables/sales.csv', header = True, inferSchema = True)
stores=spark.read.csv('/FileStore/tables/stores.csv', header = True, inferSchema = True)
features=features.drop('MarkDown1')
features=features.drop('MarkDown2')
features=features.drop('MarkDown3')
features=features.drop('MarkDown4')
features=features.drop('MarkDown5')



In [2]:
#Compare the avg sale in holidy with in normal days.

sales.filter(sales.IsHoliday=="FALSE").agg({"Weekly_Sales": "avg"}).collect()

In [3]:
#Compare the avg sale in holidy with in normal days.

sales.filter(sales.IsHoliday=="TRUE").agg({"Weekly_Sales": "avg"}).collect()

In [4]:
from pyspark.sql.functions import col
em =em.select([col(c).cast("double").alias(c) for c in em.columns])
stores =stores.select([col(c).cast("double").alias(c) for c in stores.columns])
sales =sales.select([col(c).cast("double").alias(c) for c in sales.columns])
features =features.select([col(c).cast("double").alias(c) for c in features.columns])



In [5]:
em.count

In [6]:
raw_data=em.rdd
print(raw_data)

In [7]:
from pyspark.sql import SQLContext
data = sqlContext.read.format("csv").option("header", "true").option("inferSchema", "true").load("/FileStore/tables/data.csv")
data.cache() # Cache data for faster reuse
data = data.dropna() # drop rows with missing value
type(data)

In [8]:
#find out the outlook of customer purchase quantity
display(em.select('Quantity').groupBy('Quantity').count().orderBy("count",ascending=False))

In [9]:
#find out the first 5 of product whose price is more than 10
display(em.filter(em.UnitPrice>10).limit(5))

In [10]:
#find out the most quantity
display(em.select('Quantity','Description').orderBy('Quantity',ascending=False))

In [11]:
#In this data set there is no prodct name,so we can search the total number of one product purchase based on the descripition.
display(em.select('Description').groupBy('Description').count())

In [12]:
#Since there are three seperated dataset of store sales,we can use join to view these tables and merge them.
display(stores.filter(stores.Size>100000).join(sales,sales.Store==stores.Store))


In [13]:
display(stores.join(features,features.Store==stores.Store))

In [14]:
#find out the most expemsive one
em.agg({"UnitPrice": "max"}).collect()


In [15]:
#create a temporary view table
em.createOrReplaceTempView('tmpTable')
display(spark.read.table('tmpTable'))


In [16]:
spark.sql("select * from data_csv").show()

In [17]:
#MLlib and Machine Learning;Apply a MLlib and Machine Learning analysis of your project data
from pyspark.sql.functions import col  # for indicating a column using a string in the line below
df=features
df = df.select([col(c).cast("double").alias(c) for c in df.columns])
df=df.drop("date")
df.printSchema()



In [18]:
df=df.na.drop(subset=["CPI"])
df=df.na.drop(subset=["Unemployment"])
df.where(col("Unemployment").isNull())
df.where(col("CPI").isNull())


In [19]:
df.na.drop()
train, test = df.randomSplit([0.7, 0.3])
print "We have %d training examples and %d test examples." % (train.count(), test.count())

In [20]:
display(df)


In [21]:
display(train.select("CPI", "price"))

In [22]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer
featuresCols = df.columns

featuresCols.remove("price")
# This concatenates all feature columns into a single feature vector in a new column "rawFeatures".
vectorAssembler = VectorAssembler(inputCols=featuresCols, outputCol="rawFeatures")
# This identifies categorical features and indexes them.
vectorIndexer = VectorIndexer(inputCol="rawFeatures", outputCol="features")

In [23]:
type(vectorAssembler )

In [24]:
from pyspark.ml.regression import GBTRegressor
# Takes the "features" column and learns to predict "price"
gbt = GBTRegressor(labelCol="price")

In [25]:
gbt


In [26]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
# Define a grid of hyperparameters to test:
#  - maxDepth: max depth of each decision tree in the GBT ensemble
#  - maxIter: iterations, i.e., number of trees in each GBT ensemble
paramGrid = ParamGridBuilder()\
  .addGrid(gbt.maxDepth, [2, 5])\
  .addGrid(gbt.maxIter, [10, 100])\
  .build()
# We define an evaluation metric.  This tells CrossValidator how well we are doing by comparing the true labels with predictions.
evaluator = RegressionEvaluator(metricName="rmse", labelCol=gbt.getLabelCol(), predictionCol=gbt.getPredictionCol())
# Declare the CrossValidator, which runs model tuning for us.
cv = CrossValidator(estimator=gbt, evaluator=evaluator, estimatorParamMaps=paramGrid)

In [27]:
train

In [28]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, vectorIndexer, cv])
pipelineModel = pipeline.fit(train)


In [29]:
predictions = pipelineModel.transform(test)
display(predictions.select("price", "prediction", *featuresCols))

In [30]:
display(predictions.select("price", "prediction"))

In [31]:
#Part D GraphX and GraphFrames
#A dataframe is created for the retail sale and purchase of product from one location to another. Here are the vertices:
vertices = sqlContext.createDataFrame([
("1","US",24.4039811),
("2","Canada",164.4110262),
("3","Mexico",60.75008053),
("4","S. & Cent. America",177.4399579),
("5","Europe",17.63899156),
("6","Russia",273.9722708),
("7","Other CIS",81.69126071),
("8","Iraq",177.4754949),
("9","Kuwait",103.2976087),
("10","Saudi Arabia",375.3165213),
("11","UAE",123.2311992),
("12","Other Middle East",203.159359),
("13","North Africa",58.17278392),
("14","West Africa",216.460312),
("15","East & S. Africa",6.884463733),
("16","Australasia",9.43056574),
("17","China",2.885958344),
("18","India",0.004605561),
("19","Japan",1.80022),
("20","Singapore",0.102406838),
("21","Other Asia Pacific",41.02723453)], ["id", "country", "total_value"])

In [32]:
edges = sqlContext.createDataFrame([
("1","2","selling"),
("1","3","selling"),
("1","4","selling"),
("1","8","purchase"),
("1","9","selling"),
("1","10","purchase"),
("1","14","selling"),
("1","19","purchase"),
("1","20","selling"),
("1","21","purchase"),
("2","3","selling"),
("2","4","purchase"),
("2","6","selling"),
("2","7","purchase"),
("2","8","selling"),
("2","14","selling"),
("2","19","purchase"),
("2","21","selling"),
("3","5","purchase"),
("3","8","selling"),
("3","14","purchase"),
("3","15","purchase"),
("4","5","selling"),
("4","11","purchase"),
("4","12","selling"),
("5","13","purchase"),
("5","14","purchase"),
("6","7","selling"),
("6","9","purchase"),
("6","19","purchase"),
("6","20","selling"),
("7","8","purchase"),
("7","10","purchase"),
("8","12","selling"),
("8","14","purchase"),
("9","16","purchase"),
("9","18","purchase"),
("10","20","purchase"),
("10","21","selling"),
("12","13","purchase"),
("13","17","purchase"),
("14","17","selling"),
("15","16","purchase"),
("15","19","selling"),
("16","17","purchase"),
("17","20","purchase"),
("18","20","selling"),
("18","21","purchase"),
("19","20","selling"),
("20","21","purchase")],["src", "dst", "relationship"])

In [33]:
import sys
print(sys.path)

In [34]:
from graphframes import *
g = GraphFrame(vertices, edges)
print g


In [35]:
paths = g.find("(a)-[e]->(b)")\
  .filter("e.relationship = 'export'")\
  .filter("a.total_value < b.total_value")
# The `paths` variable contains the vertex information, which we can extract:
e2 = paths.select("e.src", "e.dst", "e.relationship")

# In Spark 1.5+, the user may simplify the previous call to:
# val e2 = paths.select("e.*")

# Construct the subgraph
g2 = GraphFrame(g.vertices, e2)

In [36]:
#'g' is graph created from those vertices and edges
display(g.vertices)



In [37]:
display(g.edges)

In [38]:
display(g.degrees)

In [39]:
display(g.outDegrees)

In [40]:
display(g.inDegrees)

In [41]:
min = g.vertices.groupBy().min("total_value")
display(min)

In [42]:
max = g.vertices.groupBy().max("total_value")
display(max)

In [43]:
display(g2.vertices)

In [44]:
result = g.stronglyConnectedComponents(maxIter=10)
display(result.select("id", "component"))

In [45]:
results = g.shortestPaths(landmarks=["1", "6"])
display(results)

In [46]:
results = g.pageRank(resetProbability=0.15, tol=0.01)
display(results.vertices)

In [47]:
display(results.edges)